<a href="https://colab.research.google.com/github/thiyaneshford/LLM/blob/main/Bigquery_with_LLM_part2_workingcode_with_SQL_AGENT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user --quiet
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

!pip install -U langchain langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 69.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/6

In [11]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

In [ ]:
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
project_id = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
dataset_id = 'tg_Dataste' # @param {type:"string"}
table_name = 'LLMsales' # @param {type:"string"}
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd

In [ ]:
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

In [ ]:
def bq_qna(question):
  #create SQLDatabase instance from BQ engine
  db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=[table_name])
  # db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=['bigquery-public-data.iowa_liquor_sales.sales'])

  #create SQL DB Chain with the initialized LLM and above SQLDB instance
  db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True)


  _googlesql_prompt = """You are a BigQuery SQL expert. Given an input question, first create a syntactically correct BigQuery query to run, then look at the results of the query and return the answer to the input question.
  Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL. You can order the results to return the most informative data in the database.
  Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
  Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
  Use the following format:
  Question: "Question here"
  SQLQuery: "SQL Query to run"
  SQLResult: "Result of the SQLQuery"
  Answer: "Final answer here"
  Only use the following tables:
  {table_info}

  If someone asks for specific month, use ActivityDate between current month's start date and current month's end date

  Question: {input}"""

  BigQuerySQL_PROMPT = PromptTemplate(
      input_variables=["input", "table_info", "top_k"],
      template=_googlesql_prompt,
  )

  #passing question to the prompt template
  final_prompt = BigQuerySQL_PROMPT.format(input=question, table_info=table_name, top_k=10000)

  #pass final prompt to SQL Chain
  output = db_chain(final_prompt)


  return output['result'], output['intermediate_steps'][0]

In [ ]:
#i have installed the latest module of langchian which is stable for the creaet_SQL_Query chain
#please restart the cell after the installaiton is complete
!pip install langchain==0.0.353

  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.229
    Uninstalling langchain-0.0.229:
      Successfully uninstalled langchain-0.0.229


Using create_sql_query_chain to write queries and then using the db to run the query

In [137]:
llm

VertexAI(verbose=True, model_name='text-bison@001', client=<vertexai.language_models.TextGenerationModel object at 0x78a21d8b8730>, client_preview=<vertexai.preview.language_models._PreviewTextGenerationModel object at 0x78a21d8bbc10>, temperature=0.1, max_output_tokens=1024, top_p=0.8)

In [107]:
from langchain import SQLDatabase
#from langchain.chains import sql_database
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
PROJECT_ID = "iron-decorator-297513" # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

table_name = ['Temperature','topSpeed','Mileage']
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,

)

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)
#chain = sql_database.query.create_sql_query_chain(llm,db,k=100,prompt=)




In [ ]:
print(db.get_table_info())

NameError: ignored

In [ ]:
print(db.table_info)


CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
3 rows from Mileage table:
vehicle_number	make	type	mileage
None	None	None	None
None	None	None	None
None	None	None	None
*/


CREATE TABLE `Vehicle_ID_Table` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
3 rows from Vehicle_ID_Table table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
*/


CREATE TABLE `topSpeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
3 rows from topSpeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	100
123A	1300	1.0	100
567YZ1	1400	1.1	105
*/


In [ ]:
db.dialect

'bigquery'

In [ ]:
response = chain.invoke({"question": "How many records are there in all tables"})
print(response)

SELECT count(*) FROM Mileage


In [ ]:
response = chain.invoke({"question": "give me the vehilce id with maximum topspeed"})
print(response)

SELECT vehicle_id FROM topSpeed ORDER BY top_speed DESC LIMIT 1


In [ ]:
from langchain.prompts import PromptTemplate
from langchain_experimental.sql import SQLDatabaseChain
template = """ Given an input question, first create a yntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

'Vehicle_ID_Table','topSpeed','Mileage'

Some examples of SQL queries that correspond to questions are:
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;",
"List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make, capacitor FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_ID JOIN Topspeed ON Vehicle_ID.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_ID;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_ID);"


Question: {input}"""


PROMPT = PromptTemplate(input_variables=["input"], template=template,)



question = "give me the vehicle id with maximum topspeed"

#final_prompt = PROMPT.format(input=question, top_k=10000,dialect='bigquery')


chain = sql_database.query.create_sql_query_chain(llm, db, k=100, prompt=PROMPT)
#chain = sql_database.query.create_sql_query_chain(llm,db,k=100,prompt=final_prompt)


#chain(final_prompt)








ModuleNotFoundError: ignored

In [ ]:
response = chain.invoke({"question": "give me the vehicle id with maximum topspeed"})

In [ ]:
print(response)

SELECT vehicle_id FROM Topspeed ORDER BY top_speed DESC LIMIT 1;


In [ ]:
PROMPT

PromptTemplate(input_variables=['dialect', 'input', 'table_info', 'top_k'], template=' Given an input question, first create a yntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per BigQuery SQL\nBe careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nDatabase Description:\n\nThere are 3 tables: Vehicle_ID, Mileage, and Topspeed.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).\nThe vehicle_id column is only present i

In [ ]:
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [ ]:
print(final_prompt)

 Given an input question, first create a yntactically correct bigquery query to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query 

In [ ]:
table_name

['Vehicle_ID_Table', 'topSpeed', 'Mileage']

In [ ]:
prompt = {
    "database_description": "There are 3 tables: Vehicle_ID, Mileage, and Topspeed. To identify the vehicle_id, it needs to be joined with the vehicle_number column in other tables.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).",
    "example_queries": [
        {
            "natural_language_query": "Find the average mileage for vehicles with a top speed greater than 200 km/h.",
            "sql_query": "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;"
        },
        # ... other example queries
    ],
    "query": "Your natural language query here"
}


In [ ]:
Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Natural Language Query: "Find the average mileage for vehicles with a top speed greater than 200 km/h." SQL Query: ```sql SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;

Natural Language Query: "List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km." SQL Query: ```sql SELECT type, make, capacitor FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;

Natural Language Query: "Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." SQL Query: ```sql SELECT vehicle_id, temperature FROM Vehicle_ID JOIN Topspeed ON Vehicle_ID.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;

Natural Language Query: "Calculate the total number of vehicles with a mileage of less than 15 km/L." SQL Query: ```sql SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;

Natural Language Query: "Find the highest smoke emission value across all vehicles." SQL Query: ```sql SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_ID;

Natural Language Query: "Show the vehicle model with the lowest temperature." SQL Query: ```sql SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_ID ON Mileage.vehicle_number = Vehicle_ID.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_ID);



##using Chain methods and increased prompt with few parameters updates


In [ ]:
from langchain import SQLDatabase
#from langchain.chains import sql_database
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
PROJECT_ID = "iron-decorator-297513"
LOCATION = "us-central1"
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd


project_id = "iron-decorator-297513"
LOCATION = "us-central1"
dataset_id = 'tg_Dataste'
table_name = ['Vehicle_ID_Table','topSpeed','Mileage']
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)

In [ ]:
print(db.table_info)


CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
10 rows from Mileage table:
vehicle_number	make	type	mileage
DEF456	Honda	SUV	45
MNOP	Skoda	SUV	52
678PQR	Acura	SUV	49
456DEF	Land Rover	SUV	85
890ABCD	Mini	SUV	46
PQR678	Nissan	Van	59
JKL012	Chevrolet	Coupe	34
234VWX	Mitsubishi	Sedan	35
TUVW	Peugeot	Sedan	27
ABC123	Toyota	Sedan	78
*/


CREATE TABLE `Vehicle_ID_Table` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
10 rows from Vehicle_ID_Table table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
28	456IJKL	0.5	23
1	ABC123	0.3	32
14	QRST678	0.3	30
22	678PQR	0.3	37
30	012QRST	0.3	28
8	VWX234	0.6	29
17	123ABC	0.6	21
*/


CREATE TABLE `topSpeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
10 rows from topSpeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	1

In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True)

In [38]:
from langchain.prompts import PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate
#from langchain import SQLDatabase

template = """ Given an input question, first create a syntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

when ever the question is about Vehicle_ID first do the join with the Vehicle_ID table using Vehicle number

Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

'Vehicle_ID_Table','topSpeed','Mileage'

Some examples of SQL queries that correspond to questions are:
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;",
"List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make, capacitor FROM Mileage JOIN Vehicle_ID_Table ON Mileage.vehicle_number = Vehicle_ID_Table.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_ID_Table JOIN Topspeed ON Vehicle_ID_Table.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_ID_Table;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_ID_Table ON Mileage.vehicle_number = Vehicle_ID_Table.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_ID);"


Question: {input}"""

PROMPT = PromptTemplate(input_variables=["input"], template=template,)


In [ ]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True,prompt=PROMPT)

In [ ]:
!pip install -U langchain langchain_experimental

In [ ]:
db_chain

SQLDatabaseChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], template=' Given an input question, first create a yntactically correct bigquery  to run, then look at the results of the query and return the answer.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL\nBe careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\nwhen ever the question is about Vehicle_ID first do the join with the Vehicle_ID table using Vehicle number\n\nDatabase Description:\n\nThere are 3 tables: Vehicle_ID, Mileage, and Topspeed.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: v

In [ ]:
db_chain("give me the vehicle id with maximum Temperature")



> Entering new SQLDatabaseChain chain...
give me the vehicle id with maximum Temperature
SQLQuery:SELECT vehicle_id FROM Vehicle_ID_Table ORDER BY temperature DESC LIMIT 1;
SQLResult: [(16,)]
Answer:16
> Finished chain.


{'query': 'give me the vehicle id with maximum Temperature',
 'result': '16',
 'intermediate_steps': [{'input': 'give me the vehicle id with maximum Temperature\nSQLQuery:',
   'top_k': '5',
   'dialect': 'bigquery',
   'table_info': '\nCREATE TABLE `Mileage` (\n\t`vehicle_number` STRING, \n\t`make` STRING, \n\t`type` STRING, \n\t`mileage` INT64\n)\n\n/*\n10 rows from Mileage table:\nvehicle_number\tmake\ttype\tmileage\nDEF456\tHonda\tSUV\t45\nMNOP\tSkoda\tSUV\t52\n678PQR\tAcura\tSUV\t49\n456DEF\tLand Rover\tSUV\t85\n890ABCD\tMini\tSUV\t46\nPQR678\tNissan\tVan\t59\nJKL012\tChevrolet\tCoupe\t34\n234VWX\tMitsubishi\tSedan\t35\nTUVW\tPeugeot\tSedan\t27\nABC123\tToyota\tSedan\t78\n*/\n\n\nCREATE TABLE `Vehicle_ID_Table` (\n\t`vehicle_id` INT64, \n\t`vehicle_number` STRING, \n\t`smoke_emission` FLOAT64, \n\t`temperature` INT64\n)\n\n/*\n10 rows from Vehicle_ID_Table table:\nvehicle_id\tvehicle_number\tsmoke_emission\ttemperature\n2\tDEF456\t0.5\t28\n11\tEFGH789\t0.5\t26\n20\t012JKL\t0.5\t32

In [ ]:
db_chain("give me the vehicle id with maximum Topspeed")



> Entering new SQLDatabaseChain chain...
give me the vehicle id with maximum Topspeed
SQLQuery:SELECT vehicle_id FROM Topspeed ORDER BY top_speed DESC LIMIT 1;

DatabaseError: ignored

In [ ]:
## lets try with SQLDatabaseSequentialChain

from langchain_experimental.sql import SQLDatabaseSequentialChain

In [ ]:
#https://github.com/langchain-ai/langchain/issues/13277

In [ ]:
#Refernce :
from google.cloud import bigquery
from langchain.chains import SQLDatabaseSequentialChain
from langchain.agents import SQLDatabaseAgent

# Connect to BigQuery
client = bigquery.Client()

# Define SQL Database Agents for each table
temperature_agent = SQLDatabaseAgent(
    host="bigquery.googleapis.com",
    database="your_project_id",  # Replace with your BigQuery project ID
    user="your_service_account_email",  # If using service account authentication
    password="your_service_account_key_file"  # Path to service account key file
)

topspeed_agent = SQLDatabaseAgent(
    host="bigquery.googleapis.com",
    database="your_project_id",
    user="your_service_account_email",
    password="your_service_account_key_file"
)

mileage_agent = SQLDatabaseAgent(
    host="bigquery.googleapis.com",
    database="your_project_id",
    user="your_service_account_email",
    password="your_service_account_key_file"
)

# Function to construct the chain based on user query
def create_chain(user_query):
    chain = SQLDatabaseSequentialChain(temperature_agent)  # Start with Temperature table

    if "top speed" in user_query:
        chain.add_step("SELECT vehicle_number, top_speed FROM `your_project_id.Topspeed`")  # Replace with correct table ID
        chain.set_output_from_step(1, "vehicle_number")
    elif "mileage" in user_query:
        chain.add_step("SELECT vehicle_number, mileage FROM `your_project_id.Mileage`")  # Replace with correct table ID
        chain.set_output_from_step(1, "vehicle_number")

    chain.add_step("SELECT vehicle_id, vehicle_number, smoke_emission, temperature FROM `your_project_id.Temperature` WHERE vehicle_number = @vehicle_number")  # Join with Temperature table

    return chain

# Process user query
user_query = input("Enter your query: ")

chain = create_chain(user_query)
results = chain.run()

# Print results (consider formatting for user-friendliness)
print(results)


In [ ]:
#refernce :
def create_chain(user_query):
    chain = SQLDatabaseSequentialChain(temperature_agent)  # Start with Temperature table

    if "make" in user_query or "type" in user_query or "mileage" in user_query:  # Check for Mileage-related columns
        chain.add_step("SELECT vehicle_number, make, type, mileage FROM `your_project_id.Mileage`")
        chain.set_output_from_step(1, "vehicle_number")
    elif "capacitor" in user_query or "cc" in user_query or "top_speed" in user_query:  # Check for Topspeed-related columns
        chain.add_step("SELECT vehicle_number, capacitor, cc, top_speed FROM `your_project_id.Topspeed`")
        chain.set_output_from_step(1, "vehicle_number")

    chain.add_step("SELECT vehicle_id, vehicle_number, smoke_emission, temperature FROM `your_project_id.Temperature` WHERE vehicle_number = @vehicle_number")  # Join with Temperature table

    return chain


In [ ]:
#reference

from langchain_experimental.sql import SQLDatabaseSequentialChain
from langchain.llms import OpenAI, SQLDatabase
from langchain.schema import BasePromptTemplate

# Initialize the database (replace with your actual connection details)
db = SQLDatabase(host="your_database_host", database="your_database_name", ...)

# Initialize the language model
llm = OpenAI(...)

# Define the prompts (adjust if needed)
query_prompt = BasePromptTemplate(...)
decider_prompt = BasePromptTemplate(...)

# Create the SQLDatabaseSequentialChain
sql_seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db, query_prompt, decider_prompt)

# Define the query (replace with your actual SQL query)
query = "SELECT * FROM vehicles WHERE make = 'Toyota'"

# Use the chain
result = sql_seq_chain({"query": query})




In [ ]:
query_prompt = BasePromptTemplate(
    input_variables=["user_query"],
    output_variables=["query"],
    prompt="""Given the following user query: {user_query}
- Access the vehicle data stored in the BigQuery tables: Temperature, Topspeed, and Mileage.
- Remember that the Vehicle ID is only available in the Temperature table.
- Retrieve the necessary information, including the Vehicle ID if requested.
- Construct a SQL query that accurately fulfills the user's request."""
)


In [ ]:
decider_prompt = BasePromptTemplate(
    input_variables=["query_results"],
    output_variables=["should_continue"],
    prompt="""Analyze the results of the previous SQL query: {query_results}.
- Determine whether additional queries are necessary to fully address the user's request.
- Consider factors like:
    - Whether the Vehicle ID has been retrieved (if applicable).
    - Whether all relevant information has been gathered.
- If more queries are needed, explain what additional data should be retrieved.
- If not, indicate that the task is complete."""
)


In [152]:
query_prompt = PromptTemplate(
    input_variables=["query"],
    #output_variables=["query"],
    template="""Given the following user query: {query}
- Access the vehicle data stored in the BigQuery tables: Temperature, Topspeed, and Mileage.
- Remember that the Vehicle ID is only available in the Temperature table.
- Retrieve the necessary information, including the Vehicle ID if requested.
- Construct a SQL query that accurately fulfills the user's request."""
)


In [153]:
decider_prompt = PromptTemplate(
    input_variables=["query"],
    output_variables=["should_continue"],
    template="""Analyze the results of the previous SQL query: {query}.
- Determine whether additional queries are necessary to fully address the user's request.
- Consider factors like:
    - Whether the Vehicle ID has been retrieved (if applicable).
    - Whether all relevant information has been gathered.
- If more queries are needed, explain what additional data should be retrieved.
- If not, indicate that the task is complete."""
)


In [164]:
from langchain_experimental.sql import SQLDatabaseSequentialChain
from langchain.schema import BasePromptTemplate

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)

sql_seq_chain = SQLDatabaseSequentialChain.from_llm(llm, db,query_prompt=PROMPT)


In [165]:
sql_seq_chain

SQLDatabaseSequentialChain(decider_chain=LLMChain(prompt=PromptTemplate(input_variables=['query', 'table_names'], output_parser=CommaSeparatedListOutputParser(), template='Given the below input question and list of potential tables, output a comma separated list of the table names that may be necessary to answer this question.\n\nQuestion: {query}\n\nTable Names: {table_names}\n\nRelevant Table Names:'), llm=VertexAI(verbose=True, model_name='text-bison@001', client=<vertexai.language_models.TextGenerationModel object at 0x78a2c5f37d30>, client_preview=<vertexai.preview.language_models._PreviewTextGenerationModel object at 0x78a221a3d540>, temperature=0.1, max_output_tokens=1024, top_p=0.8), output_key='table_names'), sql_chain=SQLDatabaseChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], template='\n"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehic

In [167]:
query = "vehicle_id with maximum mileage"
result = sql_seq_chain( "vehicle_id with maximum mileage")

/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


DatabaseError: ignored

In [ ]:
## Sequential Chain

In [91]:
from langchain.chains import LLMChain, SimpleSequentialChain
template = """Your job is to find the table name based on the user query provided to you. You have three tables :
Vehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Vehicle_ID table.
when ever the user is asking for the vehicle id, you should do inner join with Vehicle_ID table using vehicle_number


% query
{user_query}

YOUR RESPONSE should look like below

result : write a sql query and if there is "vehicle_id" OR "vehicle id" in the {user_query} , replace them with vehicle_number
Impacted Table names : table names
Impacted columns : Columns and their table name

"""
prompt_template = PromptTemplate(input_variables=["user_query"], template=template)

# Holds my 'location' chain
#location_chain = LLMChain(llm=llm, prompt=prompt_template)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True,prompt=prompt_template)

In [92]:
prompt_template = PromptTemplate(input_variables=["user_query"], template=template)
first= LLMChain(llm=llm,prompt=prompt_template )

In [105]:
result = first("vehicle_id with maximum mileage")

In [106]:
print(result['text'])

result : select vehicle_number from Mileage where mileage = ( select max(mileage) from Mileage ) ; 
Impacted Table names : Mileage
Impacted columns : vehicle_number , mileage


In [79]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True,prompt=prompt_template)

In [80]:
db_chain

SQLDatabaseChain(verbose=True, llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['user_query'], template='Your job is to find the table name based on the user query provided to you. You have three tables : \nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).\nTopspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).\nMileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).\nThe vehicle_id column is only present in the Vehicle_ID table.\nwhen ever the user is asking for the vehicle id, you should do inner join with Vehicle_ID table using vehicle_number\n\n\n% query\n{user_query}\n\nYOUR RESPONSE should look like below\n\nresult : write a sql query and if there is vehicle id in the {user_query} , make appropriate join with the table using the vehicle number\nImpacted Table names :

In [81]:
from langchain.chains import LLMChain, SimpleSequentialChain
template = """ You are an SQL Expert. When there is a question asked about Vehicle_ID you should always find the Vehicle Number in the Table.
Examples :
"user_query" : "find the vehilce id with least temperature" . "SQL" : "SELECT Vehicle_Number FROM Vehicle_ID_Table ORDER BY temperature ASC LIMIT 1;"
"user_query" : "find the vehilce id with maximum Topspeed" . "SQL" : "SELECT Vehicle_Number FROM Topspeed ORDER BY temperature DESC LIMIT 1;"
"user_query" : "find the vehilce id with least temperature" . "SQL" : "SELECT Vehicle_Number FROM Vehicle_ID_Table ORDER BY temperature DESC LIMIT 1;"
"user_query" : "find the vehilce id with least temperature" . "SQL" : "SELECT Vehicle_Number FROM Vehicle_ID_Table ORDER BY temperature DESC LIMIT 1;"
"user_query" : "find the vehilce id with least temperature" . "SQL" : "SELECT Vehicle_Number FROM Vehicle_ID_Table ORDER BY temperature DESC LIMIT 1;"
"""
prompt_template = PromptTemplate(input_variables=["user_query"], template=template)

# Holds my 'location' chain
#location_chain = LLMChain(llm=llm, prompt=prompt_template)
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True,prompt=prompt_template)

In [ ]:
db_chain("vehicle id with maximum topspeed")

In [163]:
template = """ Given an input question, first create a syntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL

step 1 : Find the columns which are impacted based on the {input}. DONOT order by TABLE NAME
step 2 : If the "Vehicle_ID" is asked , It is very important to note that the "Vehicle_ID" is only present in "Temperature" Table. So do the JOIN based on the "vehicle_number"
step 3 : Create the query and check if the query is correct. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.


Database Description:

There are 3 tables: Vehicle_ID, Mileage, and Topspeed.
Temperature contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke_emission (grams/km), and temperature (degrees Celsius).
Topspeed contains: vehicle_number, capacitor (Farads), cc (engine displacement), and top_speed (km/h).
Mileage contains: vehicle_number, make (vehicle manufacturer), type (vehicle model), and mileage (km/L).
The vehicle_id column is only present in the Temperature table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
Query check: "Check if the SQLQuery is valid before executing it"
SQLResult: "Execute only if the Query check is valid"
Answer: "Final answer here"

Only use the following tables:

'Temperature','topSpeed','Mileage'

Question: {input}"""

PROMPT = PromptTemplate(input_variables=["input"], template=example+template,)

In [127]:
example = '''
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200;",
"List the vehicle model, make, and capacitor for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make, capacitor FROM Mileage JOIN Temperature ON Mileage.vehicle_number = Temperature.vehicle_number WHERE smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Temperature JOIN Topspeed ON Temperature.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Temperature;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Temperature ON Mileage.vehicle_number = Temperature.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Temperature);" '''

In [140]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, return_intermediate_steps=True,use_query_checker=True,prompt=PROMPT)

In [114]:
print(db_chain)

verbose=True llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['input'], template=' Given an input question, first create a syntactically correct bigquery  to run, then look at the results of the query and return the answer.\nUnless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL\n\nstep 1 : Find the columns which are impacted based on the {input}\nstep 2 : If the "Vehicle_ID" is asked , It is very important to note that the "Vehicle_ID" is only present in "Temperature" Table. So do the JOIN based on the "vehicle_number"\nstep 3 : Create the query and check if the query is correct. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.\n\n\nDatabase Description:\n\nThere are 3 tables: Vehicle_ID, Mileage, and Topspeed.\nVehicle_ID contains: vehicle_id (unique numerical identifier), vehicle_number (alphanumeric code), smoke

In [138]:
llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
    prompt_template = PROMPT
)

In [142]:
llm

VertexAI(verbose=True, model_name='text-bison@001', client=<vertexai.language_models.TextGenerationModel object at 0x78a2c5f37d30>, client_preview=<vertexai.preview.language_models._PreviewTextGenerationModel object at 0x78a221a3d540>, temperature=0.1, max_output_tokens=1024, top_p=0.8)

In [141]:
db_chain("vehicle_id with maximum temperature")



> Entering new SQLDatabaseChain chain...
vehicle_id with maximum temperature
SQLQuery:SELECT vehicle_id FROM Temperature ORDER BY temperature DESC LIMIT 1;

DatabaseError: ignored

In [168]:
print(db.get_table_info())


CREATE TABLE `Mileage` (
	`vehicle_number` STRING, 
	`make` STRING, 
	`type` STRING, 
	`mileage` INT64
)

/*
10 rows from Mileage table:
vehicle_number	make	type	mileage
DEF456	Honda	SUV	45
MNOP	Skoda	SUV	52
678PQR	Acura	SUV	49
456DEF	Land Rover	SUV	85
890ABCD	Mini	SUV	46
PQR678	Nissan	Van	59
JKL012	Chevrolet	Coupe	34
234VWX	Mitsubishi	Sedan	35
TUVW	Peugeot	Sedan	27
ABC123	Toyota	Sedan	78
*/


CREATE TABLE `Temperature` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
10 rows from Temperature table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
28	456IJKL	0.5	23
1	ABC123	0.3	32
14	QRST678	0.3	30
22	678PQR	0.3	37
30	012QRST	0.3	28
8	VWX234	0.6	29
17	123ABC	0.6	21
*/


CREATE TABLE `topSpeed` (
	`vehicle_number` STRING, 
	`capacitor` INT64, 
	`cc` FLOAT64, 
	`top_speed` INT64
)

/*
10 rows from topSpeed table:
vehicle_number	capacitor	cc	top_speed
YZ123	1000	1.0	100
123A	13

#### Use the SQL AGENT - The working COde

In [ ]:
#dependencies
!pip install google-cloud-aiplatform --upgrade --user --quiet
!pip install chromadb==0.3.26 pydantic==1.10.8 typing-inspect==0.8.0 typing_extensions==4.5.0 pandas datasets google-api-python-client pypdf faiss-cpu transformers config --upgrade --user --quiet


# after running restart the session


In [ ]:
# Below libraries are required to build a SQL engine for BigQuery
!pip install --user SQLAlchemy==1.4.48 --quiet
!pip install --user sqlalchemy-bigquery --quiet
# Install pandas
! pip install --user --quiet pandas

# Install HuggingFace Datasets
! pip install --user --quiet datasets

# Install Python client for Google Search API
! pip install --user --quiet google-api-python-client

# PDF loader
! pip install --user --quiet pypdf

# Install similarity search library
# TO-DO: Replace with vertex AI Matching Engine
! pip install --user --quiet faiss-cpu

!pip install -U langchain langchain_experimental

In [39]:
template = """ Given an input question, first create a syntactically correct bigquery  to run, then look at the results of the query and return the answer.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 10000 results using the LIMIT clause as per BigQuery SQL
Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

when ever the question is about Vehicle_ID first do the join with the Vehicle_ID table using Vehicle number

Database Description:

There are 3 tables: Vehicle_data, Mileage, and Topspeed.
Vehicle_data contains: vehicle_id , vehicle_number , smoke_emission, and temperature.
Topspeed contains: vehicle_number, capacitor , cc , and top_speed .
Mileage contains: vehicle_number, make , type  and mileage .
The vehicle_id column is only present in the Vehicle_data table.

The vehicle_id column is only present in the Vehicle_ID table.

Use the following format:

Question: "Question here"
SQLQuery: "SQL Query to run"
SQLResult: "Result of the SQLQuery"
Answer: "Final answer here"

Only use the following tables:

'Vehicle_data','Topspeed','Mileage'

Some examples of SQL queries that correspond to questions are:
"Find the average mileage for vehicles with a top speed greater than 200 km/h." : "SELECT Mileage.vehicle_number,Mileage.AVG(mileage) AS average_mileage FROM Mileage JOIN Topspeed ON Mileage.vehicle_number = Topspeed.vehicle_number WHERE top_speed > 200 group by Mileage.vehicle_number;",
"List the vehicle model, make for vehicles with a smoke emission between 40 and 60 grams/km.": "SELECT type, make  FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE Vehicle_data.smoke_emission BETWEEN 40 AND 60;",
"Retrieve the vehicle IDs and temperatures for vehicles with an engine displacement (cc) of 1500 or higher." : "SELECT vehicle_id, temperature FROM Vehicle_data JOIN Topspeed ON Vehicle_data.vehicle_number = Topspeed.vehicle_number WHERE cc >= 1500;",
"Calculate the total number of vehicles with a mileage of less than 15 km/L.":"SELECT COUNT(*) AS total_vehicles FROM Mileage WHERE mileage < 15;",
"Find the highest smoke emission value across all vehicles.":"SELECT MAX(smoke_emission) AS highest_smoke_emission FROM Vehicle_data;",
"Show the vehicle model with the lowest temperature.":"SELECT type AS vehicle_model FROM Mileage JOIN Vehicle_data ON Mileage.vehicle_number = Vehicle_data.vehicle_number WHERE temperature = (SELECT MIN(temperature) FROM Vehicle_data);"


Question: {input}"""

In [40]:
PROMPT = PromptTemplate(input_variables=["input"], template=template,)

In [4]:
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [41]:
from langchain import SQLDatabase
#from langchain.chains import sql_database
from google.cloud import aiplatform
from langchain.llms import VertexAI
import vertexai
from google.colab import auth as google_auth
google_auth.authenticate_user()
PROJECT_ID = "iron-decorator-297513"
LOCATION = "us-central1"
aiplatform.init(project=PROJECT_ID, location=LOCATION)
vertexai.init(project=PROJECT_ID, location=LOCATION)

from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import pandas as pd


project_id = "iron-decorator-297513"
LOCATION = "us-central1"
dataset_id = 'tg_Dataste'
table_name = ['Vehicle_data','Topspeed','Mileage']
table_uri = f"bigquery://{project_id}/{dataset_id}"
engine = create_engine(f"bigquery://{project_id}/{dataset_id}")

llm = VertexAI(
    model_name='text-bison@001',
    max_output_tokens=1024,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
    prompt_template = PROMPT
)

db = SQLDatabase(engine=engine,metadata=MetaData(bind=engine),include_tables=table_name,sample_rows_in_table_info=10)

In [8]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    top_k=20
)

In [75]:
print(agent_executor)

agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct bigquery query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 20 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (I

In [9]:
query_result = agent_executor.run(
    "Vehicle id with maxiumum temperature"
)

print("Query Result:", query_result)




> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Mileage, Vehicle_data, topSpeed
Thought:I should query the schema of the Vehicle_data table.
Action: sql_db_schema
Action Input: Vehicle_data
Observation: 
CREATE TABLE `Vehicle_data` (
	`vehicle_id` INT64, 
	`vehicle_number` STRING, 
	`smoke_emission` FLOAT64, 
	`temperature` INT64
)

/*
10 rows from Vehicle_data table:
vehicle_id	vehicle_number	smoke_emission	temperature
2	DEF456	0.5	28
11	EFGH789	0.5	26
20	012JKL	0.5	32
28	456IJKL	0.5	23
1	ABC123	0.3	32
14	QRST678	0.3	30
22	678PQR	0.3	37
30	012QRST	0.3	28
8	VWX234	0.6	29
17	123ABC	0.6	21
*/
Thought:The Vehicle_data table has the information I need. The temperature column is the relevant column.
Action: sql_db_query
Action Input: SELECT vehicle_id FROM Vehicle_data ORDER BY temperature DESC LIMIT 1
Observation: [(16,)]
Thought:I now know the final answer
Final Answer: 16

> Finished chain.
Query Result: 16


In [55]:
query_result = agent_executor.run(
    "Find the vehicle numbers and their average mileage for which the top speed is greater than 120 "
)

print("Query Result:", query_result)



> Entering new AgentExecutor chain...
Action: sql_db_list_tables
Action Input: 
Observation: Mileage, Topspeed, Vehicle_data
Thought:I should query the schema of the Mileage table to see what columns it contains.
Action: sql_db_schema Mileage
Action Input: Mileage
Observation: sql_db_schema Mileage is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:I should query the schema of the Topspeed table to see what columns it contains.
Action: sql_db_schema Topspeed
Action Input: Topspeed
Observation: sql_db_schema Topspeed is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:I should query the schema of the Vehicle_data table to see what columns it contains.
Action: sql_db_schema Vehicle_data
Action Input: Vehicle_data
Observation: sql_db_schema Vehicle_data is not a valid tool, try one of [sql_db_query, sql_db_schema, sql_db_list_tables, sql_db_query_checker].
Thought:I

KeyboardInterrupt: ignored

In [61]:
FORMAT_INSTRUCTIONS = """

Database Description:

There are 3 tables: Vehicle_data, Mileage, and Topspeed.
Vehicle_data contains: vehicle_id is the Primary key , vehicle_number , smoke_emission, and temperature.
Topspeed contains: vehicle_number is the Primary key, capacitor , cc , and top_speed .
Mileage contains: vehicle_number is the Primary key, make , type  and mileage .
The vehicle_id column is only present in the Vehicle_data table.

when ever the user is asking for the vehicle id, you should do inner join with Vehicle_ID table using vehicle_number



When ever there is Agrregate function like Average , Sum Always use the Primary key of the table along with the Select statement and group by the primary key to avoid error.
When ever there is a join , make sure to use the tablename suffix when writing the select queries to avoid ambiguity error. Example : Vehicle_data.Vehilce_number, Topspped.cc etc.



Use the following format:


Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: SQL query ONLY.
"""



In [68]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=False,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    format_instructions=FORMAT_INSTRUCTIONS,
    top_k=20,
    handle_parsing_errors=True
)

In [69]:
query_result = agent_executor.run(
    "Find the average mileage and  vehicle number with a top speed greater than 200 km/h "
)

print("Query Result:", query_result)

Query Result: SELECT AVG(Mileage.mileage) AS average_mileage, Topspeed.vehicle_number FROM Topspeed JOIN Mileage ON Topspeed.vehicle_number = Mileage.vehicle_number JOIN Vehicle_data ON Topspeed.vehicle_number = Vehicle_data.vehicle_number WHERE Topspeed.top_speed > 200 GROUP BY Topspeed.vehicle_number


In [70]:
query_result = agent_executor.run(
    "Find the Vehicle_ID with a top speed greater than 200 km/h "
)

print("Query Result:", query_result)

Query Result: SELECT Topspeed.vehicle_number FROM Topspeed JOIN Vehicle_data ON Topspeed.vehicle_number = Vehicle_data.vehicle_number WHERE Topspeed.top_speed > 200


In [71]:
query_result = agent_executor.run(
    "Find the Vehicle_ID with minimum capacitor"
)

print("Query Result:", query_result)

Query Result: SELECT vehicle_id FROM Vehicle_data WHERE vehicle_number IN (SELECT vehicle_number FROM Topspeed ORDER BY capacitor LIMIT 1)


In [72]:
query_result = agent_executor.run(
    "List the Vehicle_ID which has the mileage greater than 80"
)

print("Query Result:", query_result)

Query Result: SELECT Vehicle_data.vehicle_id FROM Vehicle_data INNER JOIN Mileage ON Vehicle_data.vehicle_number = Mileage.vehicle_number WHERE Mileage.mileage > 80


In [74]:
query_result = agent_executor.run(
    "Find the Vehicle_id that has the capacitor value greater than 1.5 and mileage greater than 35 and topspeed greater than 100"
)

print("Query Result:", query_result)

Query Result: SELECT Vehicle_data.vehicle_id FROM Vehicle_data INNER JOIN Topspeed ON Vehicle_data.vehicle_number = Topspeed.vehicle_number INNER JOIN Mileage ON Vehicle_data.vehicle_number = Mileage.vehicle_number WHERE Topspeed.capacitor > 1.5 AND Mileage.mileage > 35 AND Topspeed.top_speed > 100


In [14]:
print(agent_executor.agent)

llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], template='You are an agent designed to interact with a SQL database.\nGiven an input question, create a syntactically correct bigquery query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 20 results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELET

In [ ]:
create_sql_agent

In [ ]:
#Reference :
prompt_template = PromptTemplate(input_variables=["user_input"], template=csv_agent_prompt)

input_prompt = prompt_template.format(user_input=input)

LLM = VertexAI( model_name="text-bison@001",max_output_tokens=512,top_k=40,
                 # Adjust temperature based on your desired output characteristics
                  verbose=True,prompt=input_prompt)